In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

/mnt/data4/haryoaw_workspace/projects/2021_2/mbekk/idrecibrew2


In [2]:
from idrecibrew2.data import Seq2SeqDataFactory, Seq2SeqDataFactoryArgs
from idrecibrew2.data.indonlg_tokenizer.tokenizer import IndoNLGTokenizer
from idrecibrew2.model import LitSeq2SeqTransformers

In [3]:
import torch

In [4]:
import pandas as pd

In [5]:
from idrecibrew2.eval.training_eval import Seq2SeqTrainingEval

In [6]:
tokenizer = IndoNLGTokenizer.from_pretrained("indobenchmark/indobart-v2")


In [7]:
evaltrain = Seq2SeqTrainingEval(tokenizer)

In [8]:
df = pd.read_csv("data/processed/test.csv")

In [9]:
pd.set_option('max_colwidth', 0)

In [10]:
df.sample(2, random_state=5)

,no,src,tgt,basic_ingredient
1202,10506,baked potato mozarella with egg,2 buah kentang sedang || secukupnya parutan keju kraff || secukupnya parutan mozarella || 1 sdt margarine || sedikit lada hitam || sedikit oregano || 2 butir telur ayam kampung ||,telur
807,2620,tuna egg muffin #debm #keto,5 butir telur ayam || 6 butir telur puyuh ( bole skip kl ga suka ) || 1 kaleng tuna || 50 gram keju parut atau sesuai selera || 1 bawang bombay cincang || 1 siung bawang putih || 1 siung bawah merah || 1 batang daun bawang || italian herbs || butter untuk menumis || secukupnya garam lada dan penyedap jamur || 2 sdm santan kara ||,ikan


In [25]:
lit_model = LitSeq2SeqTransformers.load_from_checkpoint("outputs/indobert-v2/model-epoch=24-val_loss=1.731-val_bleu=26.443.ckpt")

In [116]:
list_tknz = tokenizer("nasi uduk pedas")['input_ids']

In [117]:
result = lit_model.model.generate(torch.LongTensor([list_tknz]), max_length = 500)

In [118]:
sys = tokenizer.decode(result[0])

In [119]:
print("nasi uduk pedas")
print(sys.replace("|| ", "\n"))


nasi uduk pedas
1 piring nasi uduk 
2 siung bawang merah 
3 siung bawang putih 
1 buah cabe merah besar 
10 buah cabe rawit 
2 buah cabe hijau besar 
1 / 2 buah tomat 
secukupnya garam 
secukupnya gula 
secukupnya penyedap rasa 
secukupnya kecap manis 
secukupnya air ||


In [36]:
out = [
    {
        "preds": result,
        "tgts": torch.LongTensor([tokenizer(df.iloc[1185].tgt)['input_ids']])
    }
]

In [37]:
out

[{'preds': tensor([[    2,     0,   406,   508,   322,  4498,  1869,  5148,  1320, 11042,
            8013,  3103,   406,  8101,  7737, 19955,  3103,   322,  8101,  7737,
             722,  9019,  3103,   406, 33638, 21783, 18088,  3103,   406,   508,
             647, 33638, 21783, 15194,  3103, 12956,  6229,  3103, 12956,   887,
            3103,     2]]),
  'tgts': tensor([[    0,   406,   508,   322,  4498,  3269,  4692,  3103,   406,  1782,
           39609,  3103,   406,  1782,  4480, 32742,  3103,   322, 35653,  4480,
            2074,  1496, 16680,  4635,  3103,   533, 30223,  7727,  3793,  3103,
             406,  8101, 19413,  2799,   279, 30008,  3103,   406, 30223, 13238,
           20650,  3103,   406, 33638,  4195,  4759,  3103,   406,   508,   647,
           33638,  6229,   327,  1437,  1804,  6270,   531, 12839,   388, 13238,
           20650,   329,  3103,     2]])}]

In [50]:
evaltrain.compute_eval(out)

17.35980334049656

In [83]:
evaltrain.eval_metric([df.iloc[1185].tgt], coba_lagi)

tensor(0.)

In [43]:
import sacrebleu

In [76]:
()

'1 / 2 kg daging sapi || 2 siung bawang putih || 3 siung bawang merah || 1 / 4 sdt lada hitam || secukupnya garam || secukupnya gula pasir || secukupnya minyak goreng ||'

In [46]:
skore= sacrebleu.corpus_bleu(['1 / 2 kg potato mozarella || 1 sdm tepung terigu || 2 sdm tepung tapioka || 1 sdt baking powder || 1 / 4 sdt baking soda || secukupnya garam || secukupnya air ||']
, [['1 / 2 kg daging sapi || 1 buah paprika || 1 buah bawang bombay || 2 siung bawang putih dicincang halus || 3 sachet kecap manis || 1 sdm lada hitam, haluskan || 1 sachet saus tiram || 1 sdt gula pasir || 1 / 4 sdt garam ( sedikit aja soalnya sudah asin dari saus tiram ) ||']])

In [48]:
skore.score

17.35980334049656